In [2]:
import os
import numpy as np
import open3d as o3d

def load_point_cloud(file_path):
    """
    Loads a point cloud from a file. Supports .txt, .csv (space-separated), and .ply formats.
    """
    try:
        if file_path.endswith(".ply"):
            # For .ply files, use Open3D to load the point cloud
            pcd = o3d.io.read_point_cloud(file_path)
            points = np.asarray(pcd.points)  # Extract XYZ
            if len(pcd.colors) > 0:
                colors = np.asarray(pcd.colors) * 255  # Scale RGB values to 0-255
                return np.hstack((points, colors))  # Combine XYZ and RGB
            else:
                return points  # If no colors, just return XYZ
        else:
            # For .txt or .csv files, read them directly
            return np.loadtxt(file_path)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def save_ply(file_path, points):
    """
    Saves a point cloud to a PLY file with the necessary header.
    """
    num_points = points.shape[0]
    with open(file_path, 'w') as f:
        # Write the PLY header
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {num_points}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        if points.shape[1] > 3:  # If there are colors
            f.write("property uchar red\n")
            f.write("property uchar green\n")
            f.write("property uchar blue\n")
        f.write("element face 0\n")  # Added to match File 2
        f.write("property list uchar int vertex_index\n")
        f.write("end_header\n")
        
        # Write the point data
        for point in points:
            # Format x, y, z as floats and colors as integers
            xyz = " ".join(f"{coord:.6f}" for coord in point[:3])  # XYZ coordinates
            if points.shape[1] > 3:
                rgb = " ".join(str(int(color)) for color in point[3:])  # RGB as integers
                f.write(f"{xyz} {rgb}\n")
            else:
                f.write(f"{xyz}\n")

def collect_point_clouds(folder_path, save_path):
    """
    Collects point clouds from all files in a folder and combines them into a single point cloud.
    """
    all_points = []
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            print(f"Processing: {file_name}")
            points = load_point_cloud(file_path)
            if points is not None:
                all_points.append(points)

    if all_points:
        # Combine all point clouds into a single array
        combined_point_cloud = np.vstack(all_points)
        print(f"Combined point cloud contains {combined_point_cloud.shape[0]} points.")
        
        # Save the combined point cloud as a PLY file
        save_ply(save_path, combined_point_cloud)
        print(f"Combined point cloud saved to: {save_path}")
    else:
        print("No valid point clouds were found in the folder.")

# Usage
input_folder = r"PointClouds"
output_file = r"Combined_Pointcloud.ply"  # Updated output file name
collect_point_clouds(input_folder, output_file)


Processing: NTO_test_0007_Neutral.ply
Combined point cloud contains 449506 points.
Combined point cloud saved to: Combined_Pointcloud.ply
